<a href="https://colab.research.google.com/github/GawainGan/LLM/blob/main/Module%202/Module_2_3_HuggingFace_Google_Gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Package

In [ ]:
# # Install Hugging Face Transformers and Accelerate
# !pip install -U transformers accelerate bitsandbytes

# # install the Hugging Face Hub and ipywidgets for model management and interactive widgets
# !pip install huggingface_hub
# !pip install ipywidgets

In [ ]:
!nvidia-smi # make sure that we are using the T4 GPU

Tue Jul  9 00:16:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Import packages

In [ ]:
from huggingface_hub import notebook_login
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import Markdown, display

### Hugging Face long in (Use the Granted Access Token)

In [ ]:
notebook_login()

### Load Model [Google-Gemma](https://huggingface.co/google/gemma-2-9b-it)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

### Example1: Running the model on a single / multi GPU

In [ ]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

In silicon valleys, where data flows,
A


In [ ]:
tokenizer.decode(outputs[0])

'<bos>Write me a poem about Machine Learning.\n\nIn silicon valleys, where data flows,\nA'

### Example2.1: single user interaction with prompt

In [ ]:
chat = [
    { "role": "user", "content": "Write a hello world program" },
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)
tokenizer.decode(outputs[0])

'<bos><start_of_turn>user\nWrite a hello world program<end_of_turn>\n<start_of_turn>model\n```python\nprint("Hello, world!")\n```\n\nThis program will print the text "Hello, world!" to the console. \n\nHere\'s how it works:\n\n* **`print()`** is a built-in function in Python that displays output to the console.\n* **`"Hello, world!"`** is a string literal, which is a sequence of characters enclosed in double quotes. This is the text that will be printed.\n\n\nLet me know if you\'d like to see this program in a different programming language!<end_of_turn>\n<eos>'

In [ ]:
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model
```python
print("Hello, world!")
```

This program will print the text "Hello, world!" to the console. 

Here's how it works:

* **`print()`** is a built-in function in Python that displays output to the console.
* **`"Hello, world!"`** is a string literal, which is a sequence of characters enclosed in double quotes. This is the text that will be printed.


Let me know if you'd like to see this program in a different programming language!<end_of_turn>
<eos>


### Example 2.2: single user interaction with prompt

In [ ]:
chat = [
    { "role": "user", "content": "Write me a poem about Machine Learning." },
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)
tokenizer.decode(outputs[0])

'<bos><start_of_turn>user\nWrite me a poem about Machine Learning.<end_of_turn>\n<start_of_turn>model\nIn silicon valleys, where data flows deep,\nA new intelligence, secrets to keep.\nMachine Learning, a mind of its own,\nFrom patterns and numbers, knowledge is sown.\n\nAlgorithms dance, a complex ballet,\nLearning from examples, come what may.\nNeural networks, a web intricate and vast,\nMimicking neurons, memories amassed.\n\nWith every iteration, the model improves,\nPredicting the future, unraveling truths.\nFrom image recognition to language so grand,\nMachine Learning expands, across the land.\n\nBut questions arise, as power takes hold,\nOf bias and fairness, stories untold.\nWho guides the learning, who sets the design?\nEthical dilemmas, a future to define.\n\n'

In [ ]:
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Write me a poem about Machine Learning.<end_of_turn>
<start_of_turn>model
In silicon valleys, where data flows deep,
A new intelligence, secrets to keep.
Machine Learning, a mind of its own,
From patterns and numbers, knowledge is sown.

Algorithms dance, a complex ballet,
Learning from examples, come what may.
Neural networks, a web intricate and vast,
Mimicking neurons, memories amassed.

With every iteration, the model improves,
Predicting the future, unraveling truths.
From image recognition to language so grand,
Machine Learning expands, across the land.

But questions arise, as power takes hold,
Of bias and fairness, stories untold.
Who guides the learning, who sets the design?
Ethical dilemmas, a future to define.




### Example 3: Generate text through the function

In [ ]:
def generate(input_text, system_prompt="", max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    messages = [
        {"role": "user", "content": system_prompt + '\n\n' + input_text},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=max_length, do_sample=True, temperature=0.1, top_k=50)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    text = text.replace('user\n' + system_prompt + '\n\n' + input_text + '\nmodel', '', 1)
    display(Markdown(text))

In [ ]:
generate('Once upon a time',
         system_prompt="You are Gemma, a large language model trained by Google",
         max_length=50)


Once upon a time, in a land where the sun always shone and the rivers flowed with sparkling lemonade, lived a little firefly named Flicker.  Flicker was different from all the other fireflies. While they loved to dance in the twilight